In [ ]:
from models import getModel 
from dataloader import data_pipeline, data_pipeline_with_augmentation
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B3
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.regnet import RegNetX032 
from tensorflow.keras.applications.xception import Xception 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, CSVLogger, EarlyStopping 
import pandas as pd
import numpy as np
import pickle, os, shutil
from IPython.display import clear_output

In [ ]:
preTrained = [
    ('VGG16', VGG16),
    ('Xception', Xception),
    ('MobileNetV2', MobileNetV2),
    ('ResNet50V2', ResNet50V2),
    ('RegNetX032', RegNetX032),
    ('EfficientNetV2B3', EfficientNetV2B3),
    ('EfficientNetB5', EfficientNetB5),
    ('InceptionResNetV2', InceptionResNetV2),
]
learning_rates = [
    1e-3,
    1e-3,
    1e-3,
    1e-3,
    1e-3,
    5e-6,
    5e-6,
    5e-6,
]
datasets = [
    'lastTime_balanced',
    'datasetWithGenerated',
    'classification(dilated-masks)',
    'classification(no-mask)',
    'classification(dilated-masks)(withGAN)',
]

In [ ]:
def train():
  for dataset  in datasets:
    for i, preTrainedModel in enumerate(preTrained):
        trainDataset, validateDataset = data_pipeline(DIR=dataset, BATCHSIZE=8) 
        model = getModel(preTrainedModel[1], learning_rate=learning_rates[i])

        callbacks = [ 
            EarlyStopping(patience=10, verbose=1), 
            ReduceLROnPlateau(factor=0.5, cooldown=0, patience=3, min_lr=0.5e-6), 
            CSVLogger(f'results/{dataset}/{preTrainedModel[0]}.csv'), 
            ModelCheckpoint(f'results/{dataset}/{preTrainedModel[0]}.hdf5', monitor = 'val_Accuracy', verbose = 1, mode='max', save_best_only=True) 
        ]

        model.fit(trainDataset, batch_size=8, epochs=15, verbose=1,validation_data=validateDataset, shuffle=False, callbacks=callbacks)
        model.load_weights(f'results/{dataset}/{preTrainedModel[0]}.hdf5')

        results = model.get_metrics_result()
        results = {metric:value.numpy() for metric, value in results.items()}
        results['F1_Score'] = (2*results['recall']*results['precision'])/(results['recall']+results['precision'])
        df = pd.DataFrame([results])

        pickle.dump(df.to_latex(), open(f"results/{dataset}/{preTrainedModel[0]}_classification_report_acc={np.round(results['Accuracy'], decimals=2)}.txt", 'wb'))
        clear_output(wait=True)

train() 